# Hito 3: Corona virus (Covid-19)

## Presentación

Somos el equipo conformado por Raúl Cid, José Espina, Michelle Valenzuela y Alejandro Veragua. Nuestro *dataset* corresponde al [publicado en el portal Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset) construido y mantenido por la Universidad John Hopkins, al igual que para el hito 1

## Comentarios sobre el hito 1

Se realizó un análisis exploratorio, donde descubrimos varios datos inconsistentes y outliers
Se buscaron correlaciones contra atributos como el ingreso per cápita, la temperatura, el índice de pobreza de los diferentes países, pero sin éxito. Finalmente se realizó una prueba de concepto de clústering a partir de los coeficientes de un modelo de regeresión polinomial donde sí aparecieron resultados interesantes

## Propuesta del hito 3
Considerando que los polinomios hacen "overfitting" cuando el grado es muy alto, como el ejercicio realizado en el hito 1, aprovecharemos hacer algo similar, con la diferencia de usar un modelo auto-regresivo y hacer clustering sobre los coeficientes que resulten del modelo. En particular, quisimos experimentar con un modelo auto-regresivo con media móvil (ARMA). Como se tienen datos sólo de los últimos meses y el virus es nuevo, los datos no son estacionarios (o no hay suficientes datos para demostrar que lo son), por lo que usaremos una variante llamada ARIMA (donde la "i" hace referencia a la integración), que permite encontrar las diferencias no-estacionarias para lograr la estacionariedad (https://www.statisticshowto.com/arma-model/)

## Metodología de trabajo para el hito 3
* se preprocesaron los datos, removiendo los datos outliers, ya que descubrimos en el hito 1 que hay paises con datos demasiado alejados de la tendencia central

Además, se refuerza la idea de 2 artículos publicados que usan el mismo acercamiento
1) Benvenuto, D., Giovanetti, M., Vassallo, L., Angeletti, S., & Ciccozzi, M. (2020). Application of the ARIMA model on the COVID-2019 epidemic dataset. Data in brief, 105340.
2) Yang, Q., Wang, J., Ma, H., & Wang, X. (2020). Research on COVID-19 Based on ARIMA ModelΔ—Taking Hubei, China as an example to see the epidemic in Italy. Journal of Infection and Public Health.

NOTA IMPORTANTE: Para ejecutar este notebook, se necesitan scikit, matplotlib, numpy y statsmodels. Éste último para poder aplicar el modelo ARIMA. Se debe instalar con pip en el ambiente del libro:

$ pip install statsmodels

o con conda:

$ conda install -c conda-forge statsmodels

(https://www.statsmodels.org/stable/install.html)

NOTA IMPORTANTE 2: Calcular el modelo ARIMA toma un tiempo de cómputo importante

En detalle, el plan es:

* Calcular ARIMA para todos los países


## Configuración base del notebook
El bloque a continuación configura el libro, definiendo funciones y cargando las fuentes de datos

In [2]:
# Librería de álgebra lineal
import numpy as np 
# Procesamiento de datos y carga de archivos CSV
import pandas as pd
# Librearía gráfica
import matplotlib.pylab as plt
# Librería para usar recursos del sistema operativo
import os 
# Librearía usada para extraer el nombre del archivo de cada path
import ntpath
# Librería gráfica
import seaborn as sns
# Librearía para usar herramientas relacionadas con
# aprendizaje máquina. Se usó para preprocesar y
# calcular regresión
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Colores
from matplotlib import colors
# Ticker
import matplotlib.ticker as ticker
# Animation
import matplotlib.animation as animation
# Arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
# Configuración general
%load_ext autoreload
%autoreload 2

# Matplotlib inline
%matplotlib notebook
%matplotlib inline

# Se prepara carga de los archivos del dataset
paths = []
for dirname, _, filenames in os.walk('./novel-corona-virus-2019-dataset'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        paths.append(path);
dataFrames = {}
for path in paths:
    dataFrames[ntpath.basename(path)] = pd.read_csv(path)

### Preproceso: Outliers
Como parte del preprocesamiento, se realiza el siguiente tratamiento a los datos: Para cada país, se revisan las columnas de confirmados, fallecidos y recuperados. En cada una de ellas se buscan outliers, en este caso, valores que superen en magnitud al percentil 99 o que sean menores a cero. Estos se transorman a NaN y luego interpolamos los datos para llenar estos espacios.
Además, se debe recalcular la cantidad acumulada por día para cada columna.

In [117]:
dset_base = pd.read_csv('dataset.csv')
for pais in dset_base['pais'].unique():
    for col in ['num_confirmados', 'num_fallecidos', 'num_recuperados']:
        pais_col = dset_base[dset_base['pais']==str(pais)][col]
        for value in pais_col.iteritems():
            if value[1] > pais_col.quantile(.99) or value[1] < 0:
                pais_col.loc[value[0]] = np.nan
        pais_col.interpolate(inplace=True)
        acc = pais_col.cumsum()
        dset_base[col].iloc[pais_col.index] = pais_col
        dset_base[str('acc') + col[3:]].iloc[pais_col.index] = acc

C:\Users\cabez\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\cabez\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\cabez\anaconda3\lib\site-packages\pandas\core\generic.py:7023: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


## Modelo ARIMA
Usamos un modelo ARIMA p=5,d=1,q=0. El detalle de estos parámetros se explica a continuación (basado en https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/)

* El parámetro "p" es el número de observaciones pasadas que considera el modelo. También se le llama "orden". Usamos  14 debido a que es el tiempo que demora una persona es saber que tiene el virus, considerando tiempos de síntomas y resultado del exámen. Además, 14 días es el tiempo que dura la cuarentena para las personas con resultado positivo en su exámen
* El parámetro "d" es el grado de diferenciación. Esto se usa para generar estacionariedad en series no estacionarias
* El parámetro "q" es el tamaño de la media móvil para el error de los datos anteriores

In [10]:
# Se calculan coeficientes del ARIMA por país. Si ya está calculado se lee el csv
dframe_arima = None
if os.path.isfile('arima_coefs.csv') :
    dframe_arima = pd.read_csv('arima_coefs.csv')
else :
    dset_base = pd.read_csv('dataset.csv',)
    dset_extra = pd.read_csv('atributos_extra/atributos_paises.csv')
    dset = dset_base.join(dset_extra.set_index('pais'), on='pais')
    pais_arima_coef = dict()
    paises_error = []
    for pais in dset_base.pais.unique() :
        dbset_actual = dset_base[dset_base['pais']==pais][['num_dia_desde_primer_caso','num_confirmados']]
        dbset_actual= dbset_actual.reset_index()
        series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
        series_actual.index = pd.DatetimeIndex(series_actual.index).to_period('D')
        try :
            model = ARIMA(series_actual.astype(float), order=(5,1,0),missing="drop")
            model_fit = model.fit(disp=False)
            pais_arima_coef[pais] = model_fit.params
        except :
            paises_error.append(pais)
    dframe_arima = pd.DataFrame(data=pais_arima_coef)
    dframe_arima.to_csv('arima_coefs.csv')
print(dframe_arima)

  Unnamed: 0     Japan  Mainland China  Thailand          US  Australia  \
0      const  2.837741       -2.244061  0.002067  360.064677   1.765296   
1  ar.L1.D.y -0.755774       -0.612249 -0.384722   -0.154029  -0.611496   
2  ar.L2.D.y -0.506711       -0.528041 -0.257921   -0.083624  -0.079292   
3  ar.L3.D.y -0.494117       -0.338211  0.023886   -0.313842   0.019972   
4  ar.L4.D.y -0.346803       -0.187568  0.009412   -0.272838   0.162663   
5  ar.L5.D.y -0.340686       -0.066657 -0.043951   -0.225394   0.068474   

       Brazil   Colombia  Malaysia     Mexico  ...      Laos      Mali  \
0  216.001620  61.133999  0.082935  43.268714  ... -0.012805  0.028676   
1   -0.882783  -0.833022 -0.545444  -0.397470  ... -0.823317 -0.785856   
2   -0.409989  -0.631605 -0.467962  -0.307183  ... -0.387676 -0.647222   
3   -0.518040  -0.622627 -0.218026  -0.440136  ... -0.435437 -0.546696   
4   -0.767888  -0.687674 -0.169019  -0.428673  ... -0.230595 -0.352619   
5   -0.453227  -0.468935 -0.06